In [27]:
import argparse
import pickle
from collections import Counter
#from ipdb import set_trace
#from negative_samples import negative_sampler
import numpy as np
import os
#from sma_toolkit import embeddings as emb_utils 
#import streaming_pickle as stPickle
import time
import tensorflow as tf
np.set_printoptions(threshold=np.nan)
MIN_DOC_LEN=4

In [42]:
def load_embeddings(filename,wordDict,embedding_size):
    dictionary, steps, word_embeds = pickle.load(open(filename, 'rb'))
    embedding_array = np.zeros((len(wordDict), embedding_size))
    knownWords = list(wordDict.keys())
    #print(knownWords)
    foundEmbed = 0
    for i in range(len(embedding_array)):
        index = -1
        
        w = knownWords[i]
        if w in dictionary:
            index = dictionary[w]
        elif w.lower() in dictionary:
            index = dictionary[w.lower()]
        if index >= 0:
            foundEmbed += 1
            embedding_array[i] = word_embeds[index]
        else:
            embedding_array[i] = np.random.rand(embedding_size) * 0.02 - 0.01
    #print "Found embeddings: ", foundEmbed, "/", len(knownWords)
    #print "embedding_array shape",embedding_array.shape

    return embedding_array
    

In [48]:
if __name__ == "__main__" :
    embed = "word2vec_nce.model"
    train_file = "user_text.txt"
    output = "train_embeddings.pkl"
    vocabulary_size = 100000
    min_word_freq = 5
    seed = 42
    neg_samples = 10
    t0 = time.time()
    word_counter = Counter()
    n_docs=0
    embedding_size = 128
    
    with open(train_file,"r") as fid:
        for line in fid:
            message = line.split()[1:]
            word_counter.update(message)
            n_docs+=1
    
    #keep only words that occur at least min_word_freq times
    wc = {w:c for w,c in word_counter.items() if c>min_word_freq} 
    
    #keep only the args.vocab_size most frequent words
    tw = sorted(wc.items(), key=lambda x:x[1],reverse=True)
    
    
    top_words = {w[0]:i for i,w in enumerate(tw[:vocabulary_size])}
    print(type(top_words))
    
    
    #full_E, full_wrd2idx = emb_utils.read_embeddings(args.emb,top_words)
    
    embed_matrix = load_embeddings(embed,top_words,embedding_size)
    embeddings = tf.Variable(embed_matrix, dtype=tf.float32)
    print(embeddings)
   
    
    # Start Training the model 

<class 'dict'>
<tf.Variable 'Variable_6:0' shape=(66596, 128) dtype=float32_ref>
